# H2O Models on lagged data

In [263]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML
import random

In [264]:
data = pd.read_csv("Data/20180920_mastertable.csv")
invest = data.invest
date = data.date
data = data.dropna()
#data_scaled = data.drop(["date","Unnamed: 0","return_day+1","invest"], axis=1)
data_scaled = data.drop(["date","Unnamed: 0","invest"], axis=1)
cols = data_scaled.columns
data_scaled = preprocessing.scale(data_scaled)
data_scaled = pd.DataFrame(data_scaled)
data_scaled.columns = cols
data_scaled["invest"] = invest
data_scaled["date"] = date
data = data_scaled
#data

data = pd.read_csv("Data/20180920_mastertable.csv")
for lines in range(4):
        data["lag_return_day"+str(lines+1)]=data["return_day+1"].shift(lines+1)
data

for lines in range(12):
        data["lag_return_day"+str(lines+1)]=data["return_day+1"].shift(lines+1)
        data["lag_close_day"+str(lines+1)]=data["close"].shift(lines+1)
        data["lag_volume_day"+str(lines+1)]=data["volume"].shift(lines+1)
        data["lag_googletrends_buy_sell_day"+str(lines+1)]=data["googletrends_buy_sell"].shift(lines+1)
        data["lag_Tweets (#)"+str(lines+1)]=data["Tweets (#)"].shift(lines+1)
        data["lag_Active Influencers (#)"+str(lines+1)]=data["Active Influencers (#)"].shift(lines+1)
        data["lag_Twitter Average SA"+str(lines+1)]=data["Twitter Average SA"].shift(lines+1)
        data["lag_Forum SA Merit (weighted)"+str(lines+1)]=data["Forum SA Merit (weighted)"].shift(lines+1)
        data["lag_Reddit Comments (#)"+str(lines+1)]=data["Reddit Comments (#)"].shift(lines+1)
        data["lag_Reddit Average SA"+str(lines+1)]=data["Reddit Average SA"].shift(lines+1)
        data["lag_volatility_14"+str(lines+1)]=data["volatility_14"].shift(lines+1)

In [265]:
data = data.drop(["return_day+1"], axis=1)
#data = data.drop(["lag_return_day1"], axis=1)
#data

In [266]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 15 mins
H2O cluster timezone:,Europe/Amsterdam
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,19 days
H2O cluster name:,Daniel
H2O cluster total nodes:,1
H2O cluster free memory:,1.362 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [267]:
train = data[0:260].dropna()
validation = data[262:300]
test = data[302:]
#train["invest"] = asfactor(train["invest"])
#test["invest"] = asfactor(test["invest"])
train = h2o.H2OFrame(train)
validation = h2o.H2OFrame(validation)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [268]:
x = train.columns
y = "invest"
x.remove(y)

In [269]:
# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 90,seed = 12)
aml.train(x = x, y = y,
          training_frame = train,
          validation_frame = validation,
#          leaderboard_frame = test
         )
# View the AutoML Leaderboard
lb = aml.leaderboard
lb

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,mean_per_class_error,rmse,mse
DeepLearning_grid_0_AutoML_20180920_185711_model_1,0.629824,0.711642,0.477876,0.499973,0.249973
GLM_grid_0_AutoML_20180920_185711_model_0,0.629643,0.662338,0.482301,0.485179,0.235399
DeepLearning_grid_0_AutoML_20180920_185711_model_0,0.61074,0.783749,0.444164,0.514548,0.26476
StackedEnsemble_BestOfFamily_0_AutoML_20180920_185711,0.606947,0.66863,0.486726,0.487819,0.237967
GBM_grid_0_AutoML_20180920_185711_model_5,0.600566,1.63139,0.494552,0.594907,0.353915
GBM_grid_0_AutoML_20180920_185711_model_1,0.599001,0.696337,0.451659,0.498674,0.248676
GBM_grid_0_AutoML_20180920_185711_model_3,0.593583,0.705101,0.45199,0.501186,0.251187
GBM_grid_0_AutoML_20180920_185711_model_2,0.588586,0.707151,0.45813,0.502426,0.252431
GBM_grid_0_AutoML_20180920_185711_model_0,0.569442,0.761483,0.494552,0.522037,0.272523
StackedEnsemble_AllModels_0_AutoML_20180920_185711,0.563181,0.678624,0.480254,0.492778,0.24283


In [313]:
preds = aml.leader.predict(test)
predictions = preds.as_data_frame(use_pandas=True)
#predictions

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [314]:
import sklearn
from sklearn import metrics
test_df = test.as_data_frame(use_pandas=True)
y_test = test_df[""]
#predictions = predictions.as_numpy_array()
predictions = predictions.values[:,2]
predictions
sklearn.metrics.roc_auc_score(y_test, predictions)

invest
False
True
True
True
True
False
False
False
False
False


ValueError: Expected array-like (array or non-string sequence), got 

In [195]:
for lines in range(4):
        data["lag_return_day"+str(lines+1)]=data["return_day+1"].shift(lines+1)
data

,return_day+1,close,volume,volatility_14,googletrends_buy_sell,Tweets (#),Active Influencers (#),Twitter Average SA,Forum SA Merit,Forum SA Merit (weighted),Reddit Comments (#),Reddit Average SA,invest,date,lag_return_day1,lag_return_day2,lag_return_day3,lag_return_day4
0,0.492665,-1.259410,-0.885228,-1.815638,-0.617938,-1.229630,-1.308931,2.019758,-0.730275,1.544647,-0.857860,1.610330,True,2017-05-01,NaN,NaN,NaN,NaN
1,-0.241369,-1.246628,-0.809811,-1.826866,1.350456,-1.368504,-0.838780,4.091047,-0.024282,1.840794,-0.987301,1.935256,True,2017-05-02,0.492665,NaN,NaN,NaN
2,-0.778346,-1.249417,-0.566294,-1.803027,-0.836648,-1.185774,-1.120870,1.869204,0.397697,2.109174,-0.770868,0.690981,False,2017-05-03,-0.241369,0.492665,NaN,NaN
3,0.512414,-1.263814,-0.435161,-1.564961,-1.602135,-1.448905,-1.026840,0.465855,0.413071,1.449825,-0.811220,0.253766,False,2017-05-04,-0.778346,-0.241369,0.492665,NaN
4,0.128528,-1.250772,-0.672864,-1.518604,0.326891,-1.602398,-1.967142,-0.707008,-1.211215,1.046577,-0.925463,-0.907606,True,2017-05-05,0.512414,-0.778346,-0.241369,0.492665
5,0.816963,-1.245550,-0.477814,-1.524305,1.153617,-1.719345,-2.343263,-0.836226,0.580530,1.699751,-0.995162,0.021910,True,2017-05-06,0.128528,0.512414,-0.778346,-0.241369
6,0.432548,-1.225198,-0.364056,-1.440510,-0.836648,-1.193084,-1.402961,0.895888,1.616628,2.098285,-0.897164,0.622462,True,2017-05-07,0.816963,0.128528,0.512414,-0.778346
7,0.302340,-1.212624,-0.464677,-1.427080,0.858358,-1.112683,-1.026840,-0.024230,-0.211321,1.233249,-0.688591,0.362868,True,2017-05-08,0.432548,0.816963,0.128528,0.512414
8,0.475063,-1.202752,-0.711929,-1.425187,2.225298,-1.098064,-0.932810,-0.014758,2.317732,1.980982,-0.714794,-0.162762,True,2017-05-09,0.302340,0.432548,0.816963,0.128528
9,-1.292269,-1.188450,-0.635890,-1.416988,-0.414311,-1.229630,-1.026840,-0.396239,0.478663,1.295990,-0.676014,0.871596,True,2017-05-10,0.475063,0.302340,0.432548,0.816963


In [140]:
predictions = preds.as_data_frame(use_pandas=True)

In [142]:
predictions

,predict,False,True
0,True,1.332256e-07,9.999999e-01
1,False,9.999998e-01,2.241061e-07
2,True,1.148642e-07,9.999999e-01
3,True,5.322687e-08,9.999999e-01
4,True,4.035426e-08,1.000000e+00
5,True,2.222966e-07,9.999998e-01
6,False,1.000000e+00,4.629287e-08
7,False,1.000000e+00,1.267614e-08
8,False,9.999999e-01,7.073963e-08
9,False,1.000000e+00,1.566412e-08


In [149]:
h2o.

<module 'h2o.schemas' from '/Applications/anaconda3/lib/python3.6/site-packages/h2o/schemas/__init__.py'>